# 모듈 5: 세션(Session)과 메모리(Memory) 관리

이 모듈에서는 ADK 에이전트가 대화의 맥락(Context)을 기억하고, 사용자의 정보를 유지할 수 있도록 **세션**과 **상태(State)**를 관리하는 방법을 배운다.

## 학습 목표

이 모듈을 완료하면 다음 내용을 이해할 수 있다.

- **세션과 상태**: 대화가 지속되는 동안 유지되는 데이터 저장소의 개념을 이해한다.
- **세션 생성**: `SessionService`를 사용하여 고유한 세션을 생성하고 관리한다.
- **상태 주입**: 에이전트의 프롬프트에 현재 상태(사용자 정보 등)를 동적으로 반영한다.
- **대화 기록(History)**: 대화 내용을 저장하여 에이전트가 이전 대화를 기억하게 만든다.

## 세션과 상태란 무엇인가

지금까지 만든 에이전트는 '기억 상실증'에 걸린 것처럼 매번 새로운 대화를 시작했다. 실제 서비스에서는 에이전트가 사용자의 이름이나 이전 대화 내용을 기억해야 한다.

ADK는 이를 위해 다음 개념을 제공한다.

1.  **세션(Session)**: 사용자와 에이전트 간의 연결 단위다. 고유한 `session_id`를 가진다.
2.  **상태(State)**: 세션 내부에 저장되는 키-값(Key-Value) 데이터다. 사용자 프로필, 대화 기록 등을 저장한다.
3.  **러너(Runner)**: 에이전트 실행과 세션 관리를 조율하는 관리자다.

## 환경 설정

필요한 패키지를 설치하고 환경 변수를 로드한다.

In [ ]:
# 필요한 패키지 설치
%pip install -q google-adk python-dotenv

In [1]:
import os
from dotenv import load_dotenv
from pprint import pprint
import uuid
from datetime import datetime

# 환경 변수 로드
load_dotenv()
MODEL = "gemini-2.5-flash"

True

## 예제 1: 사용자 정보를 기억하는 에이전트

첫 번째로, 사용자의 프로필(이름, 선호도)을 기억하고 이를 바탕으로 대화하는 에이전트를 만든다.

### 초기 상태(Initial State) 정의

데이터베이스에서 불러왔다고 가정하고, 에이전트가 참고할 초기 사용자 정보를 딕셔너리로 정의한다.

In [2]:
# 초기 상태 정의 (사용자 프로필)
user_profile = {
    "username": "김철수",
    "preferences": {
        "favorite_color": "파란색",
        "favorite_food": "페퍼로니 피자",
        "favorite_movie": "매트릭스",
        "hobby": "드론 날리기"
    }
}

### 에이전트 생성 (상태 주입)

에이전트의 `instruction`에 Python의 f-string을 사용하여 `user_profile` 데이터를 동적으로 삽입한다. 이렇게 하면 에이전트는 마치 사용자에 대해 이미 알고 있는 것처럼 행동한다.

In [3]:
from google.adk import Agent

# 상태 값을 프롬프트에 주입하여 에이전트 생성
profile_agent = Agent(
    name="profile_agent",
    model=MODEL,
    description="사용자의 선호도를 기억하는 에이전트",
    instruction=f"""
    당신은 사용자의 개인 정보를 잘 알고 있는 친절한 비서입니다.
    
    [사용자 정보]
    - 이름: {user_profile['username']}
    - 좋아하는 색: {user_profile['preferences']['favorite_color']}
    - 좋아하는 음식: {user_profile['preferences']['favorite_food']}
    - 좋아하는 영화: {user_profile['preferences']['favorite_movie']}
    - 취미: {user_profile['preferences']['hobby']}
    
    위 정보를 바탕으로 사용자의 질문에 답하거나 대화를 이끌어 나가세요.
    정보가 없는 내용에 대해서는 정중하게 모른다고 답하세요.
    친근한 대화체를 사용하세요.
    """
)

### Runner 및 세션 생성

`InMemoryRunner`는 내부적으로 `InMemorySessionService`를 가지고 있다. 이를 통해 세션을 생성하고 초기 상태를 설정한다.

In [4]:
from google.adk.runners import InMemoryRunner

# 1. Runner 생성
runner = InMemoryRunner(agent=profile_agent)

# 2. 세션 ID 생성
session_id = str(uuid.uuid4())

# 3. 세션 생성 (비동기 호출)
# app_name과 user_id를 키워드 인자로 명시해야 한다.
session = await runner.session_service.create_session(
    app_name="PreferenceApp",
    user_id="user_123",
    session_id=session_id,
    state=user_profile  # 초기 상태 저장
)

print(f"세션이 생성되었습니다. ID: {session_id}")

세션이 생성되었습니다. ID: 0d067979-b210-47ab-a35e-656ccc071bf1


### 에이전트 실행 (기억 테스트)

에이전트에게 사용자의 취미나 좋아하는 음식에 대해 물어본다. 에이전트는 프롬프트에 주입된 정보를 바탕으로 답변할 것이다.

In [5]:
# 사용자의 취미 물어보기
await runner.run_debug("내 취미가 뭔지 기억나?", session_id=session_id)


 ### Created new session: 0d067979-b210-47ab-a35e-656ccc071bf1

User > 내 취미가 뭔지 기억나?
profile_agent > 당연히 기억나지! 김철수 님 취미는 드론 날리기잖아. 😉 요즘 날씨도 좋은데 드론 날리러 가기 딱 좋겠다! 



[Event(model_version='gemini-2.0-flash', content=Content(
   parts=[
     Part(
       text="""당연히 기억나지! 김철수 님 취미는 드론 날리기잖아. 😉 요즘 날씨도 좋은데 드론 날리러 가기 딱 좋겠다! 
 """
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=46,
   candidates_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=46
     ),
   ],
   prompt_token_count=194,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=194
     ),
   ],
   total_token_count=240
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=-0.2678755469944166, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-8c92e8

## 예제 2: 대화형 루프와 기록(History) 관리

단순히 정보를 기억하는 것을 넘어, 사용자와 연속적으로 대화를 나누고 그 기록을 세션에 저장하는 방법을 구현한다.

### 대화 시뮬레이션

여러 턴의 대화를 시뮬레이션하면서, 각 턴마다 `runner.run_async()`를 호출하고 세션 상태를 업데이트하는 과정을 확인한다.

In [7]:
# 대화 시나리오 리스트
conversation_inputs = [
    "안녕! 오늘 점심으로 뭘 먹으면 좋을까?",  # 에이전트가 피자를 추천할 것으로 예상됨
    "그거 말고 다른 건 없을까?",              # 이전 대화(피자 추천)를 기억해야 함
    "알겠어, 그걸로 먹을게. 고마워!"
]

print("--- 대화 시작 ---")

# 대화 루프 실행
for user_input in conversation_inputs:
    # 1. run_debug 사용: 문자열 입력을 자동으로 처리하고 결과를 출력
    # 2. session_id 전달: 대화의 맥락(Context)을 유지
    # 3. 별도의 print문 불필요: run_debug가 내부적으로 User/Agent 대화를 출력
    await runner.run_debug(
        user_input, 
        session_id=session_id
    )

--- 대화 시작 ---

 ### Continue session: 0d067979-b210-47ab-a35e-656ccc071bf1

User > 안녕! 오늘 점심으로 뭘 먹으면 좋을까?
profile_agent > 안녕하세요, 김철수 님! 오늘 점심으로 뭘 먹을지 고민이시군요. 김철수 님은 페퍼로니 피자를 좋아하시니까, 맛있는 페퍼로니 피자 어떠세요? 🍕 날씨도 좋은데 피자 한 조각 들고 공원에서 드론 날리는 것도 좋을 것 같네요! 


 ### Continue session: 0d067979-b210-47ab-a35e-656ccc071bf1

User > 그거 말고 다른 건 없을까?
profile_agent > 음... 페퍼로니 피자 말고 다른 걸 추천해 달라고 하시니, 어떤 종류의 음식을 드시고 싶으신지 조금만 더 알려주시면 제가 더 잘 추천해 드릴 수 있을 것 같아요! 혹시 좋아하는 색깔처럼 파란색 음식을 드시고 싶으신 건 아니시죠? 😉 농담이에요!


 ### Continue session: 0d067979-b210-47ab-a35e-656ccc071bf1

User > 알겠어, 그걸로 먹을게. 고마워!
profile_agent > 별 말씀을요, 김철수 님! 맛있게 드시고 즐거운 하루 보내세요! 😊 혹시 매트릭스처럼 멋진 하루가 될지도 모르잖아요? 😎



### 세션 상태 확인

대화가 끝난 후, 현재 세션에 어떤 데이터가 저장되어 있는지 확인한다. `InMemoryRunner`를 사용할 경우, 기본적인 대화 맥락은 자동 관리되지만, 개발자가 직접 `session.state`에 데이터를 추가하거나 수정할 수도 있다.

In [8]:
# 현재 세션 가져오기
current_session = await runner.session_service.get_session(
    app_name="PreferenceApp",
    user_id="user_123",
    session_id=session_id
)

print("\n--- 현재 세션 상태 (State) ---")
pprint(current_session.state)


--- 현재 세션 상태 (State) ---
{'preferences': {'favorite_color': '파란색',
                 'favorite_food': '페퍼로니 피자',
                 'favorite_movie': '매트릭스',
                 'hobby': '드론 날리기'},
 'username': '김철수'}


## 상태 관리 모범 사례

1.  **초기화**: 세션 생성 시 명확한 초기 상태를 정의해야 에이전트가 오동작하지 않는다.
2.  **구조화**: 상태 데이터는 JSON 호환 딕셔너리 형태로 관리하는 것이 좋다.
3.  **식별자 관리**: `session_id`는 UUID 등을 사용하여 고유성을 보장해야 한다.
4.  **영속성**: `InMemorySessionService`는 프로그램이 종료되면 데이터가 사라진다. 실제 서비스에서는 Redis나 DB를 연동하는 `SessionService`를 구현해야 한다.

## 정리

이 모듈에서는 다음을 학습했다.
1.  **상태 주입**: f-string을 이용해 에이전트에게 사용자 정보를 인지시켰다.
2.  **세션 관리**: `InMemoryRunner`와 `SessionService`를 통해 대화의 맥락을 유지했다.
3.  **비동기 실행**: `await`를 사용하여 에이전트와 매끄럽게 상호작용했다.

이로써 ADK의 기초부터 메모리 관리까지 핵심 기능을 모두 다루었다. 이제 이 지식을 바탕으로 더욱 복잡하고 지능적인 에이전트를 구축할 수 있다.